In [ ]:
%load_ext autoreload
%autoreload 1
%aimport communicate
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
def run_traj(cycles=1, speed=0.15, amplitude=0.1, center=(1.5, 1.0)):
    """Sends commands to the cable robot to execute a circular trajectory.
    Returns: (cable lengths, cable velocities, estimated xy, setpoint xy)
    """
    period = 2 * np.pi * amplitude / speed
    with CableRobot(print_raw=False, write_timeout=None, initial_msg='d10,1') as robot:
        tstart = time.time()
        t = lambda: time.time() - tstart
        theta = lambda: 2 * np.pi * t() / period
        while True:
            robot.update()
            robot.send('ta{:.6f},{:.6f}'.format(center[0] + amplitude * np.cos(theta()),
                                        center[1] + amplitude * np.sin(theta())))
            time.sleep(0.0001)
            if (t() > period * cycles):
                break
    motor_ls = np.array([[m.length for m in datum['motors']] for datum in robot.all_data])
    motor_ldots = np.array([[m.lengthdot for m in datum['motors']] for datum in robot.all_data])
    controller_est = np.array(
        [[datum['controller'].cur_x, datum['controller'].cur_y] for datum in robot.all_data])
    controller_set = np.array(
        [[datum['controller'].set_x, datum['controller'].set_y] for datum in robot.all_data])
    robot.ser.close()
    print(f'Collected {len(robot.all_data)} samples')
    return motor_ls, motor_ldots, controller_est, controller_set

In [ ]:
# Run trajectory / collect data
ls, ldots, est, set = run_traj(speed=0.5, amplitude=0.2, center=(1.5, 1.3), cycles=1)

In [ ]:
# Plot Data
plt.figure(figsize=(8, 8))
plt.plot(*est.T)
plt.plot(*set.T)
plt.title('Example Trajectory, Gouttefarde Controller', fontsize=24)
plt.xlabel('x (m)', fontsize=24)
plt.ylabel('y (m)', fontsize=24)
plt.legend(('Estimated Position', 'Setpoint Position'))
plt.axis('equal');